In [1]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
import requests
import json
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import re
import time 

In [2]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'

response = requests.get(URL)
#type(response)
#response.status_code

In [3]:
requests.get('https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N')

<Response [200]>

In [4]:
#response.text
soup = BS(response.text)
#print(soup.prettify())

In [5]:
#reading the data into a dataframe 
TN_02=pd.read_html(str(soup.find('table')))[0]
TN_02

,Candidate,Raised,Spent,Cash on Hand,Last Report
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020


In [6]:
#keeping only the columns we want and deleting the rest 
TN_02= TN_02[['Candidate', 'Raised', 'Spent']]
TN_02

,Candidate,Raised,Spent
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488"
1,Renee Hoyos (D),"$812,784","$816,793"


In [7]:
TN_02 ['Candidate']= TN_02 ['Candidate'].astype(str)

<ipython-input-7-bdfcb1db691f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TN_02 ['Candidate']= TN_02 ['Candidate'].astype(str)


In [8]:
#splitting the candidate column into names, incumbent, winner
TN_02 [['Candidate', 'Incumbent', 'Winner']] = TN_02['Candidate'].str.split('•', 2, expand = True)

/Users/vahidehrasekhi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
#splitting candidate column into names and party 
TN_02 [['Candidate','Party']] = TN_02['Candidate'].str.split('(', expand = True)

In [10]:
TN_02

,Candidate,Raised,Spent,Incumbent,Winner,Party
0,Tim Burchett,"$1,336,276","$878,488",Incumbent,Winner,R)
1,Renee Hoyos,"$812,784","$816,793",None,None,D)


# Getting the data on other states 

In [11]:
#getting the data from britannica
URL = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'

response = requests.get(URL)
#type(response)
#response.status_code

In [12]:
requests.get('https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120')
soup = BS(response.text)

In [13]:
Congressional=pd.read_html(str(soup.find('table')))[0]
Congressional

,state,representatives
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53
5,Colorado,7
6,Connecticut,5
7,Delaware,1
8,Florida,27
9,Georgia,14


In [14]:
#dropping the 50th row from the previous cell
Congressional= Congressional [0:50] 
Congressional

,state,representatives
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53
5,Colorado,7
6,Connecticut,5
7,Delaware,1
8,Florida,27
9,Georgia,14


In [15]:
#creating a function to get data from TN02
def district_states (state, district):

    response = requests.get('https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N')
    soup = BS(response.text)
    candidates = pd.read_html(str(soup.find('table')))[0]
    candidates['state']= state
    candidates['district']= district
    
    
    return candidates 

In [16]:
district_states ('TN', '02')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020,TN,02
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020,TN,02


In [17]:
#changing the url to get the data from different states and districts 
def district_states (state, district):

    response = requests.get('https://www.opensecrets.org/races/summary?cycle=2020&id=' +state + district + '&spec=N')
    soup = BS(response.text)
    candidates = pd.read_html(str(soup.find('table')))[0]
    candidates['state']= state
    candidates['district']= district

    
    return candidates 

In [18]:
district_states ('TN', '04')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Scott Desjarlais (R) • Incumbent • Winner,"$331,464","$392,499","$302,649",12/31/2020,TN,04
1,Christopher Hale (D),"$308,731","$302,996","$5,735",12/31/2020,TN,04


In [19]:
district_states ('TN', '03')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Chuck Fleischmann (R) • Incumbent • Winner,"$1,051,653","$381,411","$1,880,341",12/31/2020,TN,03
1,Meg Gorman (D),"$85,843","$77,760","$8,083",12/31/2020,TN,03


In [20]:
district_states ('CA', '04')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Tom McClintock (R) • Incumbent • Winner,"$2,053,363","$2,107,248","$171,001",12/31/2020,CA,04
1,Brynne Kennedy (D),"$3,006,020","$3,002,532","$3,489",12/31/2020,CA,04
